In [0]:
import urllib

In [0]:
from pyspark.sql import SparkSession

In [0]:
ss = SparkSession.builder.appName('Sales_Data').getOrCreate()

In [0]:
df = spark.read.options(header='True',inferSchema='True').csv("/FileStore/tables/30cdbiotaws00104_accessKeys.csv")

In [0]:
access_key = df.head(1)[0][0] 
secret_key = df.head(1)[0][1]

In [0]:
encoded_secret_key = urllib.parse.quote(secret_key,"")
aws_bucket_name = "p4-aws-project"
mount_name = "/mnt/s3_p4"
source_url = "s3a://{0}:{1}@{2}".format(access_key,encoded_secret_key,aws_bucket_name)

# Accessing the acess key and the secret key
# Creating a source url

In [0]:
# dbutils.fs.mount(source_url,mount_name)

# Mounting S3 bucket to Databricks if True connection is established

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-45688707496529> in <cell line: 1>()
----> 1 dbutils.fs.mount(source_url,mount_name)
      2 
      3 # Mounting S3 bucket to Databricks if True connection is established

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    360                     exc.__context__ = None
    361                     exc.__cause__ = None
--> 362                     raise exc
    363 
    364             return f_with_exception_handling

ExecutionError: An error occurred while calling o539.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/s3_p4; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/s3_p4
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.s

In [0]:
from pyspark.sql.types import IntegerType, StringType, FloatType

In [0]:
# SALES - sales
# CATEGORY - categories
# MONTH - months
# PRODUCTS - products
# SUB NAMES (PRODUCT TYPE) - subs
# SUPPLIERS - suppliers
# TEMP1 - sup_pro
# TEMP2 - pro_sub
# YEAR - years
# CITY - cities

In [0]:
sales = ss.read.csv("/mnt/s3_p4/sales_ID_table.csv",inferSchema=True,header=True)

In [0]:
sales.show()

In [0]:
# START

In [0]:
categories = ss.read.csv("/mnt/s3_p4/Category(csv).csv",inferSchema=True,header=True)

In [0]:
categories = categories.withColumnRenamed("Category_id","Category_ID")

In [0]:
categories.show()

In [0]:
months = ss.read.csv("/mnt/s3_p4/Months(CSV).csv",inferSchema=True,header=True)

In [0]:
months.show()

In [0]:
products = ss.read.csv("/mnt/s3_p4/Products11.csv",inferSchema=True,header=True)

In [0]:
products.show()

In [0]:
products.count()
# Unique Products Satisfied

In [0]:
subs = ss.read.csv("/mnt/s3_p4/Sub_Names(csv).csv",inferSchema=True,header=True)

In [0]:
subs.show()

In [0]:
suppliers = ss.read.csv("/mnt/s3_p4/Suppliers(csv).csv",inferSchema=True,header=True)

In [0]:
suppliers.show()

In [0]:
suppliers.count()
# UNIQUE Supplier Satisfied

In [0]:
sup_pro = ss.read.csv("/mnt/s3_p4/Temp_1(CSV).csv",inferSchema=True,header=True)

In [0]:
sup_pro.show()

In [0]:
sup_pro.count()

In [0]:
pro_sub = ss.read.csv("/mnt/s3_p4/Temp_2(CSV).csv",inferSchema=True,header=True)

In [0]:
pro_sub.show()

In [0]:
pro_sub.count()

In [0]:
years = ss.read.csv("/mnt/s3_p4/Year(CSV).csv",inferSchema=True,header=True)

In [0]:
years.show()

In [0]:
cities = ss.read.csv("/mnt/s3_p4/City(CSV).csv",inferSchema=True,header=True)

In [0]:
cities.show()

In [0]:
# SALES - sales
# CATEGORY - categories
# MONTH - months
# PRODUCTS - products
# SUB NAMES (PRODUCT TYPE) - subs
# SUPPLIERS - suppliers
# TEMP1 - sup_pro
# TEMP2 - pro_sub
# YEAR - years
# CITY - cities

In [0]:
newdf = sales.join(cities,"City_ID","inner").select("*")

# newdf = sales & cities

# JOINED

In [0]:
newdf.show()
# newdf = sales & cities

In [0]:
newdf.select("sales_id").count()
# newdf = sales & cities

In [0]:
# SALES - sales
# CATEGORY - categories
# MONTH - months
# PRODUCTS - products
# SUB NAMES (PRODUCT TYPE) - subs
# SUPPLIERS - suppliers
# TEMP1 - sup_pro
# TEMP2 - pro_sub
# YEAR - years
# CITY - cities


In [0]:
newdf1 = products.join(pro_sub,"Product_ID","inner").select("*")

# newdf1 = products & pro_subs

In [0]:
newdf1.show()

# newdf1 = products & pro_subs

In [0]:
newdf1.count()
# newdf1 = products & pro_subs

In [0]:
newdf2 = newdf1.join(subs,"Sub_ID","inner").select("*")

#newdf2 = products & pro_subs & subs

In [0]:
# MOST IN DEMAND PRODUCTS IN EACH CATEGORY
newdf2.show()
#newdf2 = products & pro_subs & subs

In [0]:
newdf2.count()
#newdf2 = products & pro_subs & subs

In [0]:
newdf3 = sales.join(newdf2,"Product_ID","inner")

#newdf3 = (products & pro_subs & subs) & sales 

In [0]:
newdf3.printSchema()

#newdf3 = (products & pro_subs & subs) & sales 

In [0]:
newdf3.orderBy(newdf3.Product_ID.asc()).show()

#newdf3 = (products & pro_subs & subs) & sales 

In [0]:
newdf3.count()

#newdf3 = (products & pro_subs & subs) & sales 

In [0]:
newdf4 = newdf3.groupBy(newdf3.Year_ID,newdf3.Month_ID,newdf3.City_ID,newdf3.Sub_name,newdf3.Product_Name).sum("QTY")

#newdf4 = (products & pro_subs & subs) & sales .....grouped by qty

In [0]:
newdf4.display()
newdf4.count()

#newdf4 = (products & pro_subs & subs) & sales .....grouped by qty

In [0]:
newdf5 = newdf4.orderBy(newdf4['sum(QTY)'].desc())

#USER STORY 'In Demand'
#newdf5 = (products & pro_subs & subs) & sales .....grouped by qty.....ordered the same in desc order

In [0]:
newdf5.display()

#newdf5 = (products & pro_subs & subs) & sales .....grouped by qty.....ordered the same in desc order

In [0]:
newdf5.printSchema()
newdf5.count()

#newdf5 = (products & pro_subs & subs) & sales .....grouped by qty.....ordered the same in desc order

In [0]:
newdf6 = newdf5.dropDuplicates(['Year_ID','Month_ID','City_ID','Sub_name'])

# USER STORY 'In Demand part 2'
#newdf5 = (products & pro_subs & subs) & sales .....grouped by qty.....ordered the same in desc order
#newdf6 = "products" which had max sales in a particular yy,mm,city and sub

In [0]:
newdf6.display()
newdf6.count()
#newdf6 = "products" which had max sales in a particular yy,mm,city and sub

In [0]:
newdf6 = newdf6.withColumnRenamed("sum(QTY)","TOTAL_QTY")
#newdf6 = "products" which had max sales in a particular yy,mm,city and sub

In [0]:
newdf6.display()
newdf6.count()

In [0]:
newdf7 = newdf4.orderBy(newdf4['sum(QTY)'].asc())
newdf8 = newdf7.dropDuplicates(['Year_ID','Month_ID','City_ID','Sub_name'])
newdf8 = newdf8.withColumnRenamed("sum(QTY)","TOTAL_QTY")
newdf8.display()
#newdf7 = (products & pro_subs & subs) & sales .....grouped by qty.....ordered the same in asc order
#newdf8 = "products" which had min sales in a particular yy,mm,city and sub

In [0]:
new_sales = sales.withColumn("revenue",sales.QTY*sales.SP)
new_sales.display()

#new_sales has a additional column compared to sales, called revenue which is = qty * sp

In [0]:
from pyspark.sql.functions import col
city_wise_revenue = new_sales.join(cities,"City_ID","inner").groupBy("City_name").sum("revenue")
city_wise_revenue.show()
city_wise_revenue = city_wise_revenue.withColumnRenamed("sum(revenue)","revenue").sort("revenue")
city_wise_revenue.show()
min = city_wise_revenue.sort("revenue").head()
max = city_wise_revenue.sort(city_wise_revenue.revenue.desc()).head()


In [0]:
print("city " ,min[0]," has the minimum revenue, the revenue being ", min[1])
print("city " ,max[0]," has the maximum revenue, the revenue being ", max[1])